# Init for pyqcu.

In [1]:
import cupy as cp
import numpy as np
import functools
from pyqcu import define
from pyqcu import io
from pyqcu import qcu
from pyqcu import eigen, cg, bistabcg
from opt_einsum import contract
from pyqcu.set import params, argv, set_ptrs
params[define._NODE_RANK_] = define.rank
params[define._NODE_SIZE_] = define.size
kappa = 1 / (2 * argv[define._MASS_] + 8)
print('My rank is ', define.rank)
gauge_filename = f"quda_wilson-bistabcg-gauge_-{params[define._LAT_X_]}-{params[define._LAT_Y_]}-{params  [define._LAT_Z_]}-{params[define._LAT_T_]}-{params[define._LAT_XYZT_]}-{params[define._GRID_X_]}-{params[define._GRID_Y_]}-{params[define._GRID_Z_]}-{params[define._GRID_T_]}-{params[define._PARITY_]}-{params[define._NODE_RANK_]}-{params[define._NODE_SIZE_]}-{params[define._DAGGER_]}-f.h5"
print("Parameters:", params)
params[define._MG_X_] = 8
params[define._MG_Y_] = 8
params[define._MG_Z_] = 8
params[define._MG_T_] = 8


    @@@@@@######QCU NOTES START######@@@@@@@
    0. Required: MPI(e.g. 4.1.2), CUDA(e.g. 12.4), CMAKE(e.g. 3.22.1), GCC(e.g. 11.4.0), HDF5-MPI(e.g. 1.10.7,'apt install libhdf5-mpi-dev && export HDF5_MPI="ON" && pip install --no-binary=h5py h5py').
    1. The libqcu.so was compiled when pyqcu setup in download_path/PyQCU/lib, please add this path to your LD_LIBRARY_PATH.
    2. The QCU(PyQCU) splite grid by x->y->z->t, lattice by x->y->z->t->p->d->c->c or x->y->z->t->c->s(->p) and x->y->z->t->c->s->c->s(->p).
    3. The QUDA(PyQUDA) splite grid by t->z->y->x, lattice by c->c->x->y->z->t->p->d or c->s->x->y->z->t(->p) and c->s->c->s->x->y->z->t(->p).
    4. The QCU input params in numpy array(dtype=np.int32), argv in  numpy array(dtype=np.float32 or float64) array, set_ptrs in numpy array(dtype=np.int64), other in cupy array(dtype=cp.complex64 or complex128).
    5. The smallest lattice size is (x=4,y=4,z=4,t=8) that QCU support.
    @@@@@@######QCU NOTES END######@@@@@@@
    
Parameter

In [2]:
wilson_cg_params = params.copy()
wilson_cg_params[define._SET_INDEX_] = 0
wilson_cg_params[define._SET_PLAN_] = define._SET_PLAN1_
qcu.applyInitQcu(set_ptrs, wilson_cg_params, argv)

gridDim.x               :4096
blockDim.x              :128
host_params[_LAT_X_]    :16
host_params[_LAT_Y_]    :32
host_params[_LAT_Z_]    :32
host_params[_LAT_T_]    :32
host_params[_LAT_XYZT_] :524288
host_params[_GRID_X_]   :1
host_params[_GRID_Y_]   :1
host_params[_GRID_Z_]   :1
host_params[_GRID_T_]   :1
host_params[_PARITY_]   :0
host_params[_NODE_RANK_]:0
host_params[_NODE_SIZE_]:1
host_params[_DAGGER_]   :0
host_params[_MAX_ITER_] :10000
host_params[_SET_INDEX_]:0
host_params[_SET_PLAN_] :1
host_argv[_MASS_]       :0.000000e+00
host_argv[_TOL_]        :1.000000e-09
lat_2dim[_XY_]          :512
lat_2dim[_XZ_]          :512
lat_2dim[_XT_]          :512
lat_2dim[_YZ_]          :1024
lat_2dim[_YT_]          :1024
lat_2dim[_ZT_]          :1024
lat_3dim[_YZT_]         :32768
lat_3dim[_XZT_]         :16384
lat_3dim[_XYT_]         :16384
lat_3dim[_XYZ_]         :16384
lat_4dim                :524288
grid_2dim[_XY_]         :1
grid_2dim[_XZ_]         :1
grid_2dim[_XT_]         :1
grid_2

In [3]:
wilson_dslash_eo_params = params.copy()
wilson_dslash_eo_params[define._SET_INDEX_] = 1
wilson_dslash_eo_params[define._SET_PLAN_] = define._SET_PLAN0_
wilson_dslash_eo_params[define._PARITY_] = define._EVEN_
wilson_dslash_eo_params[define._DAGGER_] = define._NO_USE_
qcu.applyInitQcu(set_ptrs, wilson_dslash_eo_params, argv)

gridDim.x               :4096
blockDim.x              :128
host_params[_LAT_X_]    :16
host_params[_LAT_Y_]    :32
host_params[_LAT_Z_]    :32
host_params[_LAT_T_]    :32
host_params[_LAT_XYZT_] :524288
host_params[_GRID_X_]   :1
host_params[_GRID_Y_]   :1
host_params[_GRID_Z_]   :1
host_params[_GRID_T_]   :1
host_params[_PARITY_]   :0
host_params[_NODE_RANK_]:0
host_params[_NODE_SIZE_]:1
host_params[_DAGGER_]   :0
host_params[_MAX_ITER_] :10000
host_params[_SET_INDEX_]:1
host_params[_SET_PLAN_] :0
host_argv[_MASS_]       :0.000000e+00
host_argv[_TOL_]        :1.000000e-09
lat_2dim[_XY_]          :512
lat_2dim[_XZ_]          :512
lat_2dim[_XT_]          :512
lat_2dim[_YZ_]          :1024
lat_2dim[_YT_]          :1024
lat_2dim[_ZT_]          :1024
lat_3dim[_YZT_]         :32768
lat_3dim[_XZT_]         :16384
lat_3dim[_XYT_]         :16384
lat_3dim[_XYZ_]         :16384
lat_4dim                :524288
grid_2dim[_XY_]         :1
grid_2dim[_XZ_]         :1
grid_2dim[_XT_]         :1
grid_2

In [4]:
wilson_dslash_eo_dag_params = params.copy()
wilson_dslash_eo_dag_params[define._SET_INDEX_] = 2
wilson_dslash_eo_dag_params[define._SET_PLAN_] = define._SET_PLAN0_
wilson_dslash_eo_dag_params[define._PARITY_] = define._EVEN_
wilson_dslash_eo_dag_params[define._DAGGER_] = define._USE_
qcu.applyInitQcu(set_ptrs, wilson_dslash_eo_dag_params, argv)

gridDim.x               :4096
blockDim.x              :128
host_params[_LAT_X_]    :16
host_params[_LAT_Y_]    :32
host_params[_LAT_Z_]    :32
host_params[_LAT_T_]    :32
host_params[_LAT_XYZT_] :524288
host_params[_GRID_X_]   :1
host_params[_GRID_Y_]   :1
host_params[_GRID_Z_]   :1
host_params[_GRID_T_]   :1
host_params[_PARITY_]   :0
host_params[_NODE_RANK_]:0
host_params[_NODE_SIZE_]:1
host_params[_DAGGER_]   :1
host_params[_MAX_ITER_] :10000
host_params[_SET_INDEX_]:2
host_params[_SET_PLAN_] :0
host_argv[_MASS_]       :0.000000e+00
host_argv[_TOL_]        :1.000000e-09
lat_2dim[_XY_]          :512
lat_2dim[_XZ_]          :512
lat_2dim[_XT_]          :512
lat_2dim[_YZ_]          :1024
lat_2dim[_YT_]          :1024
lat_2dim[_ZT_]          :1024
lat_3dim[_YZT_]         :32768
lat_3dim[_XZT_]         :16384
lat_3dim[_XYT_]         :16384
lat_3dim[_XYZ_]         :16384
lat_4dim                :524288
grid_2dim[_XY_]         :1
grid_2dim[_XZ_]         :1
grid_2dim[_XT_]         :1
grid_2

In [5]:
wilson_dslash_oe_params = params.copy()
wilson_dslash_oe_params[define._SET_INDEX_] = 3
wilson_dslash_oe_params[define._SET_PLAN_] = define._SET_PLAN0_
wilson_dslash_oe_params[define._PARITY_] = define._ODD_
wilson_dslash_oe_params[define._DAGGER_] = define._NO_USE_
qcu.applyInitQcu(set_ptrs, wilson_dslash_oe_params, argv)

gridDim.x               :4096
blockDim.x              :128
host_params[_LAT_X_]    :16
host_params[_LAT_Y_]    :32
host_params[_LAT_Z_]    :32
host_params[_LAT_T_]    :32
host_params[_LAT_XYZT_] :524288
host_params[_GRID_X_]   :1
host_params[_GRID_Y_]   :1
host_params[_GRID_Z_]   :1
host_params[_GRID_T_]   :1
host_params[_PARITY_]   :1
host_params[_NODE_RANK_]:0
host_params[_NODE_SIZE_]:1
host_params[_DAGGER_]   :0
host_params[_MAX_ITER_] :10000
host_params[_SET_INDEX_]:3
host_params[_SET_PLAN_] :0
host_argv[_MASS_]       :0.000000e+00
host_argv[_TOL_]        :1.000000e-09
lat_2dim[_XY_]          :512
lat_2dim[_XZ_]          :512
lat_2dim[_XT_]          :512
lat_2dim[_YZ_]          :1024
lat_2dim[_YT_]          :1024
lat_2dim[_ZT_]          :1024
lat_3dim[_YZT_]         :32768
lat_3dim[_XZT_]         :16384
lat_3dim[_XYT_]         :16384
lat_3dim[_XYZ_]         :16384
lat_4dim                :524288
grid_2dim[_XY_]         :1
grid_2dim[_XZ_]         :1
grid_2dim[_XT_]         :1
grid_2

In [6]:
wilson_dslash_oe_dag_params = params.copy()
wilson_dslash_oe_dag_params[define._SET_INDEX_] = 4
wilson_dslash_oe_dag_params[define._SET_PLAN_] = define._SET_PLAN0_
wilson_dslash_oe_dag_params[define._PARITY_] = define._ODD_
wilson_dslash_oe_dag_params[define._DAGGER_] = define._USE_
qcu.applyInitQcu(set_ptrs, wilson_dslash_oe_dag_params, argv)

gridDim.x               :4096
blockDim.x              :128
host_params[_LAT_X_]    :16
host_params[_LAT_Y_]    :32
host_params[_LAT_Z_]    :32
host_params[_LAT_T_]    :32
host_params[_LAT_XYZT_] :524288
host_params[_GRID_X_]   :1
host_params[_GRID_Y_]   :1
host_params[_GRID_Z_]   :1
host_params[_GRID_T_]   :1
host_params[_PARITY_]   :1
host_params[_NODE_RANK_]:0
host_params[_NODE_SIZE_]:1
host_params[_DAGGER_]   :1
host_params[_MAX_ITER_] :10000
host_params[_SET_INDEX_]:4
host_params[_SET_PLAN_] :0
host_argv[_MASS_]       :0.000000e+00
host_argv[_TOL_]        :1.000000e-09
lat_2dim[_XY_]          :512
lat_2dim[_XZ_]          :512
lat_2dim[_XT_]          :512
lat_2dim[_YZ_]          :1024
lat_2dim[_YT_]          :1024
lat_2dim[_ZT_]          :1024
lat_3dim[_YZT_]         :32768
lat_3dim[_XZT_]         :16384
lat_3dim[_XYT_]         :16384
lat_3dim[_XYZ_]         :16384
lat_4dim                :524288
grid_2dim[_XY_]         :1
grid_2dim[_XZ_]         :1
grid_2dim[_XT_]         :1
grid_2

In [7]:
print("Set pointers:", set_ptrs)
print("Set pointers data:", set_ptrs.data)

Set pointers: [94255414169152 94255423958464 94255424132544 94255424309072
 94255424447344              0              0              0
              0              0]
Set pointers data: <memory at 0x7efee56331c0>


# Read from hdf5 files.

In [8]:
print("Gauge filename:", gauge_filename)
gauge = io.hdf5_xxxtzyx2grid_xxxtzyx(params, gauge_filename)
fermion_in_filename = gauge_filename.replace("gauge", "fermion-in")
print("Fermion in filename:", fermion_in_filename)
fermion_in = io.hdf5_xxxtzyx2grid_xxxtzyx(
    params, fermion_in_filename)
fermion_out_filename = gauge_filename.replace("gauge", "fermion-out")
print("Fermion out filename:", fermion_out_filename)
quda_fermion_out = io.hdf5_xxxtzyx2grid_xxxtzyx(
    params, fermion_out_filename)
fermion_out = cp.zeros_like(fermion_in)
print("Fermion out data:", fermion_out.data)
print("Fermion out shape:", fermion_out.shape)
# eigenvalues_filename = gauge_filename.replace("gauge", "eigenvalues")
# print("Eigenvalues filename:", eigenvalues_filename)
# eigenvalues = io.hdf5_xxx2xxx(file_name=eigenvalues_filename)
# print("Eigenvalues data:", eigenvalues.data)
# print("Eigenvalues shape:", eigenvalues.shape)
eigenvectors_filename = gauge_filename.replace("gauge", "eigenvectors")
print("Eigenvectors filename:", eigenvectors_filename)
eigenvectors = io.eigenvectors2esctzyx(
    params=params, eigenvectors=io.hdf5_xxx2xxx(file_name=eigenvectors_filename))
print("Eigenvectors data:", eigenvectors.data)
print("Eigenvectors shape:", eigenvectors.shape)

Gauge filename: quda_wilson-bistabcg-gauge_-32-32-32-32-1048576-1-1-1-1-0-0-1-0-f.h5
Grid Index T: 0, Grid Index Z: 0, Grid Index Y: 0, Grid Index X: 0
Grid Lat T: 32, Grid Lat Z: 32, Grid Lat Y: 32, Grid Lat X: 16
All Dest Shape: (3, 3, 4, 2, 32, 32, 32, 16)
Dest Shape: (3, 3, 4, 2, 32, 32, 32, 16)
Fermion in filename: quda_wilson-bistabcg-fermion-in_-32-32-32-32-1048576-1-1-1-1-0-0-1-0-f.h5
Grid Index T: 0, Grid Index Z: 0, Grid Index Y: 0, Grid Index X: 0
Grid Lat T: 32, Grid Lat Z: 32, Grid Lat Y: 32, Grid Lat X: 16
All Dest Shape: (2, 4, 3, 32, 32, 32, 16)
Dest Shape: (2, 4, 3, 32, 32, 32, 16)
Fermion out filename: quda_wilson-bistabcg-fermion-out_-32-32-32-32-1048576-1-1-1-1-0-0-1-0-f.h5
Grid Index T: 0, Grid Index Z: 0, Grid Index Y: 0, Grid Index X: 0
Grid Lat T: 32, Grid Lat Z: 32, Grid Lat Y: 32, Grid Lat X: 16
All Dest Shape: (2, 4, 3, 32, 32, 32, 16)
Dest Shape: (2, 4, 3, 32, 32, 32, 16)
Fermion out data: <MemoryPointer 0xb28400000 device=0 mem=<cupy.cuda.memory.PooledMemor

# Run wilson bistabcg from pyqcu test.

In [9]:
qcu.applyWilsonBistabCgQcu(fermion_out, fermion_in,
                           gauge, set_ptrs, wilson_cg_params)
# qcu.applyWilsonCgQcu(fermion_out, fermion_in,
#                            gauge, set_ptrs, wilson_cg_params)
print("Fermion out data:", fermion_out.data)
print("Fermion out shape:", fermion_out.shape)
print("QUDA Fermion out data:", quda_fermion_out.data)
print("QUDA Fermion out shape:", quda_fermion_out.shape)
print("Difference:", cp.linalg.norm(fermion_out -
      quda_fermion_out)/cp.linalg.norm(quda_fermion_out))

##RANK:0##LOOP:118##Residual:(Fermion out data: <MemoryPointer 0xb28400000 device=0 mem=<cupy.cuda.memory.PooledMemory object at 0x7efe77312330>>
Fermion out shape: (2, 4, 3, 32, 32, 32, 16)
QUDA Fermion out data: <MemoryPointer 0xb22400000 device=0 mem=<cupy.cuda.memory.PooledMemory object at 0x7efe773032f0>>
QUDA Fermion out shape: (2, 4, 3, 32, 32, 32, 16)
2.27222e-10,1.97371e-23i)
multi-gpu wilson bistabcg total time: (without malloc free memcpy) :1.736276480 sec
######TIME  :4182.25######
##RANK      :0
##LOOP      :999
##tmp0      :(1.03257e-11,2.49512e-12i)
##tmp1      :(4.79284e-12,-2.12052e-23i)
##rho_prev  :(-2.31288e-06,4.83391e-06i)
##rho       :(-2.31288e-06,4.83391e-06i)
##alpha     :(0.629024,-0.434716i)
##beta      :(0.059529,-0.0243195i)
##omega     :(2.1544,0.520593i)
##send_tmp  :(0.00984323,0i)
##norm2_tmp :(4.97484e+07,0.000224118i)
##diff_tmp  :(1.9786e-10,-8.91365e-22i)
##lat_4dim  :(524288,0i)
Difference: 3.056118e-07


# Give CG & BISTABCG Dslash.
> src_o-set_ptr->kappa()**2*dslash_oe(dslash_eo(src_o))

In [10]:
def cg_dslash_no_dag(src):
    tmp0 = cp.zeros_like(src)
    tmp1 = cp.zeros_like(src)
    qcu.applyWilsonDslashQcu(
        tmp0, src, gauge, set_ptrs, wilson_dslash_eo_params)
    qcu.applyWilsonDslashQcu(
        tmp1, tmp0, gauge, set_ptrs, wilson_dslash_oe_params)
    return src-kappa**2*tmp1


def cg_dslash_dag(src):
    tmp0 = cp.zeros_like(src)
    tmp1 = cp.zeros_like(src)
    qcu.applyWilsonDslashQcu(
        tmp0, src, gauge, set_ptrs, wilson_dslash_eo_dag_params)
    qcu.applyWilsonDslashQcu(
        tmp1, tmp0, gauge, set_ptrs, wilson_dslash_oe_dag_params)
    return src-kappa**2*tmp1


def cg_dslash(src):
    return cg_dslash_dag(cg_dslash_no_dag(src))

def bistabcg_dslash(src):
    return cg_dslash_no_dag(src)



# Give matvec.

In [11]:
def matvec(src):
    return cg_dslash(src)

# Run matvec(eigenvector[.]) ?= eigenvalue[.]*eigenvector[.] for eigen test. (pass, don't run this)

In [12]:
# for i, ev in enumerate(eigenvalues):
#     print(f"λ_{i} = {ev:.2e}")
#     # Verify eigenvector
#     v = eigenvectors[i]
#     w = cp.zeros_like(v)
#     w = cg_dslash(v)
#     error = cp.linalg.norm(w - ev * v) / cp.linalg.norm(w)
#     print(f"Relative error: {error:.2e}")
#     j = i+1
#     if j == len(eigenvalues):
#         j = 0
#     print(
#         f"Diff between λ_{i} and λ_{j}: {cp.linalg.norm(eigenvectors[i] - eigenvectors[j])/cp.linalg.norm(eigenvectors[i]):.2e}")

# Give guage's eigenvalues and eigenvectors to hdf5 files. (pass, don't run this)

In [13]:
# eigen_solver = eigen.solver(
#     n=params[define._LAT_XYZT_] * define._LAT_HALF_SC_, k=define._LAT_Ne_,matvec=cg_dslash,dtype=gauge.dtype)
# eigenvalues, eigenvectors = eigen_solver.run()
# io.xxx2hdf5_xxx(
#     eigenvalues, params, gauge_filename.replace("gauge", "eigenvalues"))
# io.xxx2hdf5_xxx(
#     eigenvectors, params, gauge_filename.replace("gauge", "eigenvectors"))

# Origin CG. (pass, don't run this)

In [14]:
# b_e = fermion_in[define._EVEN_].flatten()
# b_o = fermion_in[define._ODD_].flatten()
# b__o = cp.zeros_like(b_o)
# tmp = cp.zeros_like(b_o)
# # b__o=b_o+kappa*D_oe(b_e)
# qcu.applyWilsonDslashQcu(tmp, b_e, gauge, set_ptrs, wilson_dslash_oe_params)
# b__o = b_o+kappa*tmp
# # b__o -> Dslash^dag b__o
# b__o = cg_dslash_dag(b__o)
# # Dslash(x_o)=b__o
# cg_solver = cg.slover(b=b__o, matvec=cg_dslash, tol=1e-10, max_iter=1000000)
# x_o = cg_solver.run()
# # x_e  =b_e+kappa*D_eo(x_o)
# qcu.applyWilsonDslashQcu(tmp, x_o, gauge, set_ptrs, wilson_dslash_eo_params)
# x_e = b_e+kappa*tmp
# # give qcu_fermion_out
# qcu_fermion_out = cp.zeros_like(quda_fermion_out)
# qcu_fermion_out[define._EVEN_] = x_e.reshape(
#     quda_fermion_out[define._EVEN_].shape)
# qcu_fermion_out[define._ODD_] = x_o.reshape(
#     quda_fermion_out[define._ODD_].shape)
# print(np.linalg.norm(qcu_fermion_out-quda_fermion_out) /
#       np.linalg.norm(quda_fermion_out))

# Origin BISTABCG. (pass, don't run this)

In [15]:
# b_e = fermion_in[define._EVEN_].flatten()
# b_o = fermion_in[define._ODD_].flatten()
# b__o = cp.zeros_like(b_o)
# tmp = cp.zeros_like(b_o)
# # b__o=b_o+kappa*D_oe(b_e)
# qcu.applyWilsonDslashQcu(tmp, b_e, gauge, set_ptrs, wilson_dslash_oe_params)
# b__o = b_o+kappa*tmp
# # Dslash(x_o)=b__o
# bistabcg_solver = bistabcg.slover(
#     b=b__o, matvec=bistabcg_dslash, tol=1e-10, max_iter=1000000)
# x_o = bistabcg_solver.run()
# # x_e  =b_e+kappa*D_eo(x_o)
# qcu.applyWilsonDslashQcu(tmp, x_o, gauge, set_ptrs, wilson_dslash_eo_params)
# x_e = b_e+kappa*tmp
# # give qcu_fermion_out
# qcu_fermion_out = cp.zeros_like(quda_fermion_out)
# qcu_fermion_out[define._EVEN_] = x_e.reshape(
#     quda_fermion_out[define._EVEN_].shape)
# qcu_fermion_out[define._ODD_] = x_o.reshape(
#     quda_fermion_out[define._ODD_].shape)
# print(np.linalg.norm(qcu_fermion_out-quda_fermion_out) / \
#     np.linalg.norm(quda_fermion_out))

# Give guage's orth_eigenvectors to hdf5 files. (pass, don't run this)

In [16]:
# _eigenvectors = io.xxxtzyx2mg_xxxtzyx(input_array=eigenvectors, params=params)
# _eigenvectors.shape  # escTtZzYyXx
# def orthogonalize(eigenvectors):
#     _eigenvectors = eigenvectors.copy()
#     size_e, size_s, size_c, size_T, size_t, size_Z, size_z, size_Y, size_y, size_X, size_x = eigenvectors.shape
#     print(size_e, size_s, size_c, size_T, size_t,
#           size_Z, size_z, size_Y, size_y, size_X, size_x)
#     for T in range(size_T):
#         for Z in range(size_Z):
#             for Y in range(size_Y):
#                 for X in range(size_X):
#                     origin_matrix = eigenvectors[:,
#                                                  :, :, T, :, Z, :, Y, :, X, :]
#                     _shape = origin_matrix.shape
#                     _origin_matrix = origin_matrix.reshape(size_e, -1)
#                     condition_number = np.linalg.cond(_origin_matrix.get())
#                     print(f"矩阵条件数: {condition_number}")
#                     a = _origin_matrix[:, 0]
#                     b = _origin_matrix[:, -1]
#                     print(cp.dot(a.conj(), b))
#                     Q = cp.linalg.qr(_origin_matrix.T)[0]
#                     condition_number = np.linalg.cond(Q.get())
#                     print(f"矩阵条件数: {condition_number}")
#                     a = Q[:, 0]
#                     b = Q[:, -1]
#                     print(cp.dot(a.conj(), b))
#                     _eigenvectors[:, :, :, T, :, Z, :, Y, :, X, :] = Q.T.reshape(
#                         _shape)
#     return _eigenvectors
# orth_eigenvectors = orthogonalize(_eigenvectors)
# io.xxx2hdf5_xxx(
#     orth_eigenvectors, params, gauge_filename.replace("gauge", "orth_eigenvectors"))

# MultiGrid - give grids.

In [17]:
orth_eigenvectors_filename = gauge_filename.replace("gauge", "orth_eigenvectors")
print("Orth orth_eigenvectors filename:", orth_eigenvectors_filename)
orth_eigenvectors = io.eigenvectors2esctzyx(
    params=params, eigenvectors=io.hdf5_xxx2xxx(file_name=orth_eigenvectors_filename))
print("Orth orth_eigenvectors data:", orth_eigenvectors.data)
print("Orth orth_eigenvectors shape:", orth_eigenvectors.shape)
testvectors = io.xxxtzyx2mg_xxxtzyx(input_array=orth_eigenvectors, params=params)
_src = io.xxxtzyx2mg_xxxtzyx(
    input_array=fermion_in[define._EVEN_], params=params)
# _src = io.xxxtzyx2mg_xxxtzyx(
#     input_array=fermion_out[define._EVEN_], params=params)

Orth orth_eigenvectors filename: quda_wilson-bistabcg-orth_eigenvectors_-32-32-32-32-1048576-1-1-1-1-0-0-1-0-f.h5
Dest Shape: (24, 4, 3, 8, 4, 8, 4, 8, 4, 8, 2)
Orth orth_eigenvectors data: <MemoryPointer 0xb7f400000 device=0 mem=<cupy.cuda.memory.PooledMemory object at 0x7efe75cb38f0>>
Orth orth_eigenvectors shape: (24, 4, 3, 32, 32, 32, 16)
Input Array Shape: (24, 4, 3, 32, 32, 32, 16)
Dest Shape: (24, 4, 3, 8, 4, 8, 4, 8, 4, 8, 2)
Input Array Shape: (4, 3, 32, 32, 32, 16)
Dest Shape: (4, 3, 8, 4, 8, 4, 8, 4, 8, 2)


<!-- # MultiGrid - R*vector.
![](./image0-dev40.png) -->

In [18]:
r_src = _src


def r_vec(src):
    return contract("escTtZzYyXx,scTtZzYyXx->eTZYX", testvectors, src)


r_dest = r_vec(r_src)

In [19]:
r_dest.shape

(24, 8, 8, 8, 8)

<!-- # MultiGrid - P*vector.
![](./image1-dev40.png) -->


In [20]:
p_src = r_dest


def p_vec(src):
    return contract("escTtZzYyXx,eTZYX->scTtZzYyXx", cp.conj(testvectors), src)


p_dest = p_vec(p_src)

In [21]:
p_dest.shape

(4, 3, 8, 4, 8, 4, 8, 4, 8, 2)

<!-- # MultiGrid - verify above.
![](./image2-dev40.png) -->

In [22]:
print(cp.linalg.norm(r_src))
print(cp.linalg.norm(p_dest))

3547.24
3532.7056


In [23]:
print(cp.linalg.norm(r_src-p_dest)/cp.linalg.norm(r_src))

0.09043299


In [24]:
print(cp.linalg.norm(r_src-p_vec(r_vec(r_src)))/cp.linalg.norm(r_src))

0.09043299


In [25]:
r_src.flatten()[:50]

array([1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j,
       1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j,
       1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j,
       1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j,
       1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j,
       1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j, 1.+1.j,
       1.+1.j, 1.+1.j], dtype=complex64)

In [26]:
p_dest.flatten()[:50]

array([1.167576  +0.9505424j , 1.1282876 +0.9586141j ,
       1.0756463 +0.8200437j , 1.058318  +0.91000664j,
       0.9969356 +1.068906j  , 0.9797176 +1.0625488j ,
       0.93820333+1.0070767j , 0.76377463+1.1238567j ,
       1.1066835 +1.1300876j , 0.8434196 +1.0795238j ,
       0.7763571 +1.0511982j , 0.81844413+0.84909153j,
       0.78963894+1.0923208j , 1.0337933 +0.8952743j ,
       0.9988575 +0.93025064j, 1.0472151 +1.0066209j ,
       1.0446063 +1.0375896j , 0.892465  +0.9763166j ,
       1.0538195 +1.0875429j , 0.8134135 +1.0639153j ,
       0.9023303 +0.9247432j , 0.981052  +1.0085366j ,
       0.8793721 +1.2040772j , 0.78277594+1.0184716j ,
       1.031603  +0.95662045j, 0.89636445+1.0806143j ,
       0.94076717+0.93580586j, 1.1366302 +1.0215473j ,
       0.8842316 +1.1014208j , 0.83240545+1.0948608j ,
       0.9384273 +0.96103394j, 1.0361595 +1.0604932j ,
       1.0288321 +0.95639586j, 0.9762008 +1.1355324j ,
       0.73028743+1.0778768j , 1.1808312 +1.0461483j ,
       0.9

In [27]:
cp.linalg.norm(r_src-p_dest)/cp.linalg.norm(r_src)

array(0.09043299, dtype=float32)

In [28]:
cp.linalg.norm(r_src-p_dest)/cp.linalg.norm(p_dest)

array(0.09080505, dtype=float32)

In [29]:
p_vec(r_vec(p_vec(r_vec(p_vec(r_vec(p_vec(r_vec(r_src)))))))).flatten()[:50]

array([1.1675767 +0.9505423j , 1.128288  +0.95861405j,
       1.075647  +0.8200431j , 1.0583186 +0.9100065j ,
       0.996936  +1.0689057j , 0.9797179 +1.0625486j ,
       0.9382022 +1.0070776j , 0.76377344+1.1238575j ,
       1.1066833 +1.1300873j , 0.843419  +1.0795233j ,
       0.77635676+1.0512j    , 0.8184434 +0.8490934j ,
       0.7896385 +1.0923212j , 1.0337931 +0.89527476j,
       0.9988583 +0.93025076j, 1.0472162 +1.0066211j ,
       1.0446074 +1.0375894j , 0.892466  +0.97631633j,
       1.0538201 +1.0875425j , 0.81341416+1.063915j  ,
       0.9023303 +0.9247428j , 0.981052  +1.0085362j ,
       0.87937105+1.2040778j , 0.7827748 +1.0184724j ,
       1.0316026 +0.9566202j , 0.8963641 +1.0806139j ,
       0.940767  +0.93580747j, 1.1366295 +1.0215489j ,
       0.8842313 +1.1014209j , 0.83240527+1.094861j  ,
       0.9384279 +0.961034j  , 1.0361606 +1.0604935j ,
       1.0288332 +0.9563956j , 0.9762019 +1.1355324j ,
       0.730288  +1.0778766j , 1.1808316 +1.0461485j ,
       0.9

In [30]:
cp.linalg.norm(r_src-p_vec(r_vec(p_vec(r_vec(p_vec(r_vec(p_vec(r_vec(r_src)))))))))/cp.linalg.norm(r_src) #???

array(0.09043298, dtype=float32)

In [31]:
# _mat = contract("escTtZzYyXx,escTtZzYyXx->scTtZzYyXx",
#                 testvectors, cp.conj(testvectors)).flatten()
# print(cp.linalg.norm(_mat))
# print(_mat[:100])

# MultiGrid - R*matvec\*P.

In [32]:
def _r_matvec_p(src, matvec):
    return r_vec(matvec(p_vec(io.xxx2eTZYX(src, params))))


def r_matvec_p(src, matvec):
    return io.array2xxx(_r_matvec_p(src, matvec))

# MultiGrid - verify above.

In [33]:
D_r_src = matvec(r_src)

multi-gpu wilson dslash total time: (without malloc free memcpy) :0.001983511 sec
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.001859857 sec
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.001906338 sec
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.001850431 sec


In [34]:
D_r_src.flatten()[:50]

array([ 0.09512203-0.15065098j,  0.02459466-0.10281426j,
        0.14802423-0.04478627j,  0.05378214-0.09193704j,
       -0.08474712+0.0208632j , -0.06649091+0.06668942j,
       -0.03820895+0.03143284j, -0.06110204+0.1998555j ,
       -0.09466349-0.10954653j, -0.09903856+0.15328293j,
       -0.11581516+0.23665299j,  0.15433274+0.0833789j ,
       -0.11228501+0.18325019j,  0.07416645+0.00099519j,
        0.05309803-0.03660914j,  0.034646  -0.03932991j,
       -0.00817882-0.03089109j,  0.03485045+0.13626698j,
       -0.12274602-0.06901661j, -0.10658678+0.15540114j,
        0.09019466+0.07479304j,  0.00329622+0.01262958j,
       -0.2735389 +0.08964469j, -0.02719212+0.18662325j,
        0.07507145-0.0618236j , -0.09112616+0.10951558j,
        0.08583128+0.02523299j, -0.0734987 -0.1731855j ,
       -0.09989857+0.04082112j, -0.1259279 +0.17906842j,
        0.00141903+0.04172344j, -0.08249421-0.07952813j,
        0.06730361-0.03395787j, -0.16046013+0.01053373j,
       -0.04409004+0.2682615j ,

In [ ]:
p_r_D_p_r_dest = p_vec(_r_matvec_p(r_vec(r_src), matvec=cg_dslash))

Input Array Shape: (24, 8, 8, 8, 8)
Dest Shape: (24, 8, 8, 8, 8)
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.036014747 sec
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.007829612 sec
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.001877931 sec
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.007824046 sec


In [55]:
p_r_D_p_r_dest.flatten()[:50]

array([ 2.8194975e-02+0.01779438j,  1.9536421e-02+0.01874043j,
        2.6979309e-02-0.00564249j,  1.5606621e-02-0.00971756j,
        1.2855723e-02+0.01153428j,  1.3916093e-02+0.00912649j,
       -2.1622099e-02+0.02844416j, -2.3205638e-02+0.02636695j,
       -1.2025104e-02-0.0043727j , -8.9504719e-03+0.0031735j ,
        1.0010808e-02+0.00924149j,  1.8886885e-02-0.00021074j,
        6.3355304e-03-0.00064231j,  2.3217805e-02+0.00153761j,
        8.2115643e-05+0.01287063j,  1.7011244e-02+0.01075634j,
        2.9906198e-02+0.01418719j,  1.8058531e-02+0.01371316j,
        3.2366689e-02-0.00604446j,  1.7136531e-02-0.00260097j,
        9.4566504e-03+0.01958975j,  1.1899084e-02+0.01579495j,
       -1.1868380e-02+0.03067987j, -1.2942081e-02+0.02253125j,
       -1.2726447e-02-0.00657428j, -5.8244430e-03-0.00102587j,
        9.1750175e-05+0.01301394j,  1.1270541e-02+0.00761647j,
        5.9559247e-03-0.00201995j,  1.7859971e-02+0.00487764j,
       -1.0425074e-02+0.0156606j ,  9.8662656e-03+0.012

In [37]:
cp.linalg.norm(D_r_src-p_r_D_p_r_dest)/cp.linalg.norm(D_r_src)

array(1.0020683, dtype=float32)

# MultiGrid - CG (BUG!!!)

In [40]:
# b_e = fermion_in[define._EVEN_].flatten()
# b_o = fermion_in[define._ODD_].flatten()
# b__o = cp.zeros_like(b_o)
# tmp = cp.zeros_like(b_o)
# # b__o=b_o+kappa*D_oe(b_e)
# qcu.applyWilsonDslashQcu(tmp, b_e, gauge, set_ptrs, wilson_dslash_oe_params)
# b__o = b_o+kappa*tmp
# # b__o -> Dslash^dag b__o
# b__o = cg_dslash_dag(b__o)

In [41]:
# # # Dslash(x_o)=b__o
# # cg_solver = cg.slover(b=b__o, matvec=cg_dslash, tol=1e-10, max_iter=1000000)
# # x_o = cg_solver.run()

# # mg version
# mg_b__o = r_vec(io.xxxtzyx2mg_xxxtzyx(io.fermion2sctzyx(b__o, params), params)).flatten()
# cg_solver = cg.slover(b=mg_b__o, matvec=functools.partial(r_matvec_p, matvec=cg_dslash),
#                       tol=1e-5, max_iter=1000000)
# mg_x_o = cg_solver.run()

In [42]:
# # x_e  =b_e+kappa*D_eo(x_o)
# qcu.applyWilsonDslashQcu(tmp, x_o, gauge, set_ptrs, wilson_dslash_eo_params)
# x_e = b_e+kappa*tmp
# # give qcu_fermion_out
# qcu_fermion_out = cp.zeros_like(quda_fermion_out)
# qcu_fermion_out[define._EVEN_] = x_e.reshape(
#     quda_fermion_out[define._EVEN_].shape)
# qcu_fermion_out[define._ODD_] = x_o.reshape(
#     quda_fermion_out[define._ODD_].shape)


In [43]:
# print(np.linalg.norm(qcu_fermion_out-quda_fermion_out) / \
#     np.linalg.norm(quda_fermion_out))

# MultiGrid - BISTABCG (TESTING......)

In [44]:
b_e = fermion_in[define._EVEN_].flatten()
b_o = fermion_in[define._ODD_].flatten()
b__o = cp.zeros_like(b_o)
tmp = cp.zeros_like(b_o)
# b__o=b_o+kappa*D_oe(b_e)
qcu.applyWilsonDslashQcu(tmp, b_e, gauge, set_ptrs, wilson_dslash_oe_params)
b__o = b_o+kappa*tmp

multi-gpu wilson dslash total time: (without malloc free memcpy) :0.001906390 sec


In [45]:
# # Dslash(x_o)=b__o
# bistabcg_solver = bistabcg.slover(
#     b=b__o, matvec=bistabcg_dslash, tol=1e-10, max_iter=1000000)
# x_o = bistabcg_solver.run()
# io.xxx2hdf5_xxx(x_o, params, 'x_o.h5')

In [46]:
# mg version
mg_b__o = r_vec(io.xxxtzyx2mg_xxxtzyx(io.fermion2sctzyx(b__o, params), params)).flatten()
bistabcg_solver = bistabcg.slover(
    b=mg_b__o, matvec=functools.partial(r_matvec_p, matvec=bistabcg_dslash), tol=1e-10, max_iter=1000000)
mg_x_o = bistabcg_solver.run()
_x_o=io.array2xxx(p_vec(io.xxx2eTZYX(mg_x_o, params)))
io.xxx2hdf5_xxx(_x_o, params, '_x_o.h5')


Input Array Shape: (4, 3, 32, 32, 32, 16)
Dest Shape: (4, 3, 8, 4, 8, 4, 8, 4, 8, 2)
Input Array Shape: (98304,)
Dest Shape: (24, 8, 8, 8, 8)
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.002097293 sec
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.001883254 sec
Input Array Shape: (98304,)
Dest Shape: (24, 8, 8, 8, 8)
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.045845481 sec
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.001890147 sec
Input Array Shape: (98304,)
Dest Shape: (24, 8, 8, 8, 8)
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.039266133 sec
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.001975546 sec
Iteration 0: Residual = 4.937418e+07, Time = 1.486281 s
Input Array Shape: (98304,)
Dest Shape: (24, 8, 8, 8, 8)
multi-gpu wilson dslash total time: (without malloc free memcpy) :0.036784753 sec
multi-gpu wilson dslash total time: (without malloc free 

In [47]:
# # x_e  =b_e+kappa*D_eo(x_o)
# qcu.applyWilsonDslashQcu(tmp, x_o, gauge, set_ptrs, wilson_dslash_eo_params)
# x_e = b_e+kappa*tmp
# # give qcu_fermion_out
# qcu_fermion_out = cp.zeros_like(quda_fermion_out)
# qcu_fermion_out[define._EVEN_] = x_e.reshape(
#     quda_fermion_out[define._EVEN_].shape)
# qcu_fermion_out[define._ODD_] = x_o.reshape(
#     quda_fermion_out[define._ODD_].shape)
# print(np.linalg.norm(qcu_fermion_out-quda_fermion_out) / \
#     np.linalg.norm(quda_fermion_out))

In [48]:
x_o=io.hdf5_xxx2xxx(params,'x_o.h5')
_x_o=io.hdf5_xxx2xxx(params,'_x_o.h5')

Dest Shape: (6291456,)
Dest Shape: (6291456,)


In [49]:
x_o.flatten()[:50]

array([74.27604 +92.02284j , 75.06629 +87.020996j, 63.950462+81.42317j ,
       71.90431 +81.19496j , 82.32697 +77.19529j , 82.58721 +76.42146j ,
       75.92041 +76.61929j , 83.29001 +63.37289j , 90.26471 +89.01645j ,
       85.437584+67.62439j , 82.527954+61.06994j , 66.877235+63.152378j,
       86.607956+60.486282j, 69.97304 +78.853806j, 74.98956 +78.34286j ,
       82.30312 +82.18618j , 80.42873 +82.24485j , 75.13653 +69.27514j ,
       84.342064+79.84675j , 83.38979 +62.740913j, 70.93342 +69.10556j ,
       78.41749 +76.8612j  , 92.75483 +71.3723j  , 76.95801 +63.98175j ,
       76.83701 +84.04088j , 86.23326 +71.82796j , 74.12482 +74.47032j ,
       80.77271 +88.2747j  , 88.081   +68.75456j , 86.93835 +63.496307j,
       76.293976+74.005066j, 85.7808  +80.60354j , 73.14485 +80.3956j  ,
       86.552826+75.09205j , 84.15033 +56.783974j, 82.566315+92.946976j,
       80.47373 +77.690834j, 69.55831 +75.1817j  , 69.10493 +87.87933j ,
       78.504234+86.637924j, 85.1525  +73.68484j , 

In [50]:
_x_o.flatten()[:50]

array([67.387924+54.741905j, 65.105705+55.079216j, 61.63377 +47.820557j,
       61.62418 +52.77228j , 58.587605+63.035408j, 56.884136+62.6757j  ,
       55.78987 +58.38521j , 45.477867+65.36519j , 65.18134 +66.84824j ,
       49.72288 +63.687965j, 45.12284 +61.752632j, 47.732132+50.282852j,
       46.197052+63.797306j, 59.563198+51.779213j, 58.285847+51.587685j,
       60.263958+56.143394j, 60.08226 +59.86721j , 51.5574  +56.209953j,
       60.115032+63.66837j , 47.312984+61.629982j, 53.40726 +54.270096j,
       57.201862+59.00563j , 51.886227+69.46479j , 46.15534 +58.915108j,
       61.128544+56.918156j, 52.94934 +63.996147j, 54.918674+54.924637j,
       66.50179 +59.966515j, 51.692146+64.842514j, 48.013412+63.798714j,
       55.602352+53.35087j , 60.25013 +59.350258j, 59.330276+55.18667j ,
       56.654083+65.25717j , 41.943768+62.61409j , 69.07778 +60.58457j ,
       58.46769 +60.54844j , 54.74251 +51.766106j, 63.747993+51.31568j ,
       62.206665+60.188053j, 54.819607+62.899696j, 

In [51]:
print(np.linalg.norm(_x_o-x_o) /
      np.linalg.norm(x_o))

0.28642112


# End for CG & BISTABCG. (pass, don't run this)

In [52]:
# cg_solver.end()
# bistabcg_solver.end()

# End for pyqcu. (pass, don't run this)

In [53]:
# qcu.applyEndQcu(set_ptrs, params)
# qcu.applyEndQcu(set_ptrs, wilson_dslash_eo_params)
# qcu.applyEndQcu(set_ptrs, wilson_dslash_oe_params)
# qcu.applyEndQcu(set_ptrs, wilson_dslash_eo_dag_params)
# qcu.applyEndQcu(set_ptrs, wilson_dslash_oe_dag_params)